# Obtain Data of AGN from SIMBAD

The code to obtain the data is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView)

Here we use [Astroquery](https://astroquery.readthedocs.io/en/latest/index.html) package to retriev the data.

In [1]:
from datetime import date

import pyvo
import numpy as np
from astroquery.ned import Ned

import astropy.units as u
from astropy.table import hstack
from astropy.coordinates import SkyCoord

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
	,ra
	,dec
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [2]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We save the date in a variable for use later in the file names of the results

In [3]:
today = date.today()

The sample results is saved in a .csv file:

In [4]:
simbad_sample_Sy = result.to_table()
simbad_sample_Sy.write(today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.csv',
                    format='ascii.csv',overwrite=True)

Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is because when we search the information in [NED](https://ned.ipac.caltech.edu/) using the object id that we obtained from SIMBAD, sometimes it doesn't reconigze it, so we will use coordinates instead

In [32]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])

In [29]:
result_table = Ned.query_region(cat_smb_sample[0],radius=0.001 * u.deg)

print(result_table) # an astropy.table.Table


No.        Object Name            RA     ... Diameter Points Associations
                               degrees   ...                             
--- ------------------------- ---------- ... --------------- ------------
  1 WISEA J140931.24+051131.6  212.38024 ...               4            0


In [30]:
obj_name=result_table["Object Name"]
phot_table = Ned.get_table(obj_name, table='photometry')

In [34]:
phot_table["Frequency"]

1950000000000000.0
1950000000000000.0
1950000000000000.0
1950000000000000.0
1290000000000000.0
1290000000000000.0
1290000000000000.0
1290000000000000.0
836000000000000.0
836000000000000.0
836000000000000.0
